In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("qoute_dataset.csv")

In [4]:
df.head()

,quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe


In [5]:
df['quote'][0]

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

In [6]:
df.shape

(3038, 2)

In [7]:
quotes=df['quote']
quotes.head()

,quote
0,“The world as we have created it is a process ...
1,"“It is our choices, Harry, that show what we t..."
2,“There are only two ways to live your life. On...
3,"“The person, be it gentleman or lady, who has ..."
4,"“Imperfection is beauty, madness is genius and..."


In [8]:
quotes=quotes.str.lower()

In [9]:
import string
translator=str.maketrans("","",string.punctuation)
quotes=quotes.apply(lambda x:x.translate(translator))

In [10]:
quotes.head()

,quote
0,“the world as we have created it is a process ...
1,“it is our choices harry that show what we tru...
2,“there are only two ways to live your life one...
3,“the person be it gentleman or lady who has no...
4,“imperfection is beauty madness is genius and ...


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [12]:
vocab_size=10000
tokenizer=Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(quotes)

In [13]:
word_index=tokenizer.word_index
print(len(word_index))
list(word_index.items())[:10]

8978


[('the', 1),
 ('you', 2),
 ('to', 3),
 ('and', 4),
 ('a', 5),
 ('i', 6),
 ('is', 7),
 ('of', 8),
 ('that', 9),
 ('it', 10)]

In [14]:
sequence=tokenizer.texts_to_sequences(quotes)

In [15]:
for i in range(3):
  print(quotes[i])

“the world as we have created it is a process of our thinking it cannot be changed without changing our thinking”
“it is our choices harry that show what we truly are far more than our abilities”
“there are only two ways to live your life one is as though nothing is a miracle the other is as though everything is a miracle”


In [16]:
for i in range(3):
  print(sequence[i])

[713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70, 293, 10, 145, 12, 809, 104, 752, 70, 2461]
[947, 7, 70, 871, 373, 9, 433, 21, 19, 465, 14, 294, 52, 54, 70, 3676]
[1337, 14, 53, 201, 714, 3, 81, 15, 36, 37, 7, 29, 329, 93, 7, 5, 1157, 1, 101, 7, 29, 329, 126, 7, 5, 3677]


In [17]:

X = []
y = []

for seq in sequence:
  for i in range(1,len(seq)):
    input_seq = seq[:i]
    output_seq = seq[i]
    X.append(input_seq)
    y.append(output_seq)

In [18]:


len(X)

85271

In [19]:


len(y)

85271

In [20]:


max_len = max(len(x) for x in X)
print(max_len)

745


In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_padded = pad_sequences(X, maxlen=max_len, padding='pre')


In [22]:
y = np.array(y)

In [23]:

X_padded.shape

(85271, 745)

In [24]:

from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y, num_classes=vocab_size)


In [25]:

y.shape


(85271,)

In [26]:


y_one_hot.shape

(85271, 10000)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,LSTM, Dense


In [28]:

embedding_dim = 50
rnn_units = 128

In [29]:

rnn_model = Sequential()

rnn_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
rnn_model.add(SimpleRNN(units=rnn_units))
rnn_model.add(Dense(units=vocab_size, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [30]:
rnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [31]:

rnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:


lstm_model = Sequential()
lstm_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
lstm_model.add(LSTM(units=rnn_units))
lstm_model.add(Dense(units=vocab_size, activation='softmax'))

In [33]:

lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [34]:

lstm_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [35]:
epochs=10
batch_size=128

In [36]:
history_rnn=rnn_model.fit(
    X_padded, y_one_hot,
    epochs=epochs,
    batch_size=batch_size,

)

Epoch 1/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 47s 62ms/step - accuracy: 0.0377 - loss: 7.0975
Epoch 2/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.0623 - loss: 6.2525
Epoch 3/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.0813 - loss: 5.9817
Epoch 4/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 40s 61ms/step - accuracy: 0.1062 - loss: 5.6209
Epoch 5/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.1216 - loss: 5.3477
Epoch 6/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 40s 61ms/step - accuracy: 0.1376 - loss: 5.0786
Epoch 7/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.1524 - loss: 4.8330
Epoch 8/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 41s 61ms/step - accuracy: 0.1716 - loss: 4.5950
Epoch 9/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 40s 61ms/step - accuracy: 0.1944 - loss: 4.3721
Epoch 10/10
667/667 ━━━━━━━━━━━━━━━━━━━━ 40s 60ms/step - accuracy: 0.2202 - loss: 4.1597


In [37]:
epochs=100
bactch_size=128
history_lstm=lstm_model.fit(
    X_padded, y_one_hot,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1

)

Epoch 1/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 36s 51ms/step - accuracy: 0.0364 - loss: 7.1148 - val_accuracy: 0.0437 - val_loss: 6.6816
Epoch 2/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.0524 - loss: 6.3610 - val_accuracy: 0.0665 - val_loss: 6.5726
Epoch 3/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.0761 - loss: 6.0702 - val_accuracy: 0.0881 - val_loss: 6.4788
Epoch 4/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.0941 - loss: 5.8367 - val_accuracy: 0.0969 - val_loss: 6.4310
Epoch 5/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.1095 - loss: 5.6480 - val_accuracy: 0.0999 - val_loss: 6.4122
Epoch 6/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.1147 - loss: 5.4936 - val_accuracy: 0.1039 - val_loss: 6.4244
Epoch 7/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 52ms/step - accuracy: 0.1267 - loss: 5.3198 - val_accuracy: 0.1069 - val_loss: 6.4476
Epoch 8/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.1343 - loss: 5

In [38]:
lstm_model.save('lstm_model.h5')

In [39]:
index_to_word = {}
for word, index in word_index.items():
  index_to_word[index] = word


In [40]:
index_to_word

{1: 'the',
 2: 'you',
 3: 'to',
 4: 'and',
 5: 'a',
 6: 'i',
 7: 'is',
 8: 'of',
 9: 'that',
 10: 'it',
 11: 'in',
 12: 'be',
 13: 'not',
 14: 'are',
 15: 'your',
 16: 'have',
 17: 'for',
 18: 'but',
 19: 'we',
 20: 'if',
 21: 'what',
 22: 'with',
 23: 'all',
 24: 'love',
 25: 'can',
 26: 'my',
 27: 'when',
 28: 'will',
 29: 'as',
 30: 'who',
 31: 'do',
 32: 'or',
 33: 'me',
 34: 'he',
 35: 'they',
 36: 'life',
 37: 'one',
 38: 'was',
 39: 'like',
 40: 'there',
 41: 'people',
 42: 'on',
 43: 'its',
 44: 'at',
 45: 'so',
 46: 'never',
 47: 'no',
 48: 'them',
 49: 'dont',
 50: 'know',
 51: 'just',
 52: 'more',
 53: 'only',
 54: 'than',
 55: 'because',
 56: 'this',
 57: 'want',
 58: 'up',
 59: 'how',
 60: 'his',
 61: 'things',
 62: 'world',
 63: 'by',
 64: 'think',
 65: 'make',
 66: 'about',
 67: 'time',
 68: 'from',
 69: 'always',
 70: 'our',
 71: 'an',
 72: 'out',
 73: 'us',
 74: 'good',
 75: 'said',
 76: 'she',
 77: 'her',
 78: 'way',
 79: 'go',
 80: 'am',
 81: 'live',
 82: 'has',
 83:

In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [42]:
def predictor(model,tokenizer,text,max_len):
  text = text.lower()

  seq = tokenizer.texts_to_sequences([text])[0]
  seq = pad_sequences([seq], maxlen=max_len, padding='pre')

  pred = model.predict(seq,verbose = 0)
  pred_index = np.argmax(pred)
  return index_to_word[pred_index]

In [46]:

seed_text = "what are you"
next_word = predictor(lstm_model,tokenizer,seed_text,max_len)
print(next_word)

implying


In [54]:
def generate_text(model,tokenizer,seed_text,max_len,n_words):
  for _ in range(n_words):
    next_word = predictor(model,tokenizer,seed_text,max_len)
    if next_word == "":
      break
    seed_text += " " + next_word
  return seed_text

In [55]:
seed = "the meaning of life"
generate_text = generate_text(lstm_model,tokenizer,seed,max_len,10)
print(generate_text)


the meaning of life is to be by being caring about that we have


In [57]:

import pickle
with open("tokenizer.pkl", "wb") as f:
  pickle.dump(tokenizer, f)

In [52]:

with open("max_len.pkl", "wb") as f:
  pickle.dump(max_len, f)